# Big project: Patient-nurse-room assignment

In this project, you are required to solve a healthcare scheduling problem involving the assignment of patients to rooms and nurses to rooms in a hospital. The goal is to develop an optimization model using mip that minimizes the total delay in patient admissions while adhering to a set of operational and resource constraints. This problem is inspired by the Integrated Healthcare Timetabling Competition 2024 (https://ihtc2024.github.io/).

The hospital operates with a set of patients, rooms, and nurses, each with specific characteristics. Patients are defined by their release date, indicating the earliest day they can be admitted, and a due date, specifying the latest allowable admission day. Patients also have a required length of stay. Rooms are characterized by fixed capacities that determine the maximum number of patients they can accommodate at any given time. Additionally, some patients may have incompatibilities with certain rooms, restricting their assignment.

Nurses have predefined rosters detailing their availability across days. Each occupied room must have one assigned nurse on every day of the scheduling horizon. Nurses can be assigned to a maximum of 3 rooms in a given day.

The problem requires assigning each patient to a room and ensuring that every occupied room has an assigned nurse during each day. These assignments must satisfy several constraints: no room may exceed its capacity, nurse assignments must respect their availability, and patients must only be assigned to compatible rooms. The primary objective is to minimize the total delay in admissions, defined as the sum of the differences between actual admission dates and the release dates of all patients. In what follows we detail the constraints and decisions.


**Constraints:**

- A patient’s length of stay must be respected, meaning they occupy a room for the full duration of their stay.
- A room cannot exceed its capacity on any given day.
- Patients can only be assigned to rooms that they are compatible with.
- Each occupied room must have exactly one assigned nurse per day.
- Nurses can only be assigned to rooms on days when they are available.
- Nurses can be assigned to a maximum of 3 different rooms in the same day.
- Admission days must be within the specified release and due dates.

**Decisions:**

- Assign each patient to a room, ensuring that the same room is assigned for the duration of their stay.
- Determine the admission day for each patient within the allowed range of their release and due dates.
- Assign one nurse to each occupied room on every day of the scheduling period.

The objective is to minimize the total admission delay, defined as the sum of the differences between the admission day and the release date of patients.


**Sumbmission instructions:**

- Each group can be formed by maximum 3 students
- One sumbission per group
- Submission consists of uploading a .ipynb file with the following name format: "LastName1_LastName2_LastName3.ipynb" where the name of the file contains the last names of the students in alphabetic order.
- The submitted file must contain:
  - Name of students
  - Python code using mip library with the model and print out of the objective value in one single code cell
- Questions regarding the project description will be answered in the Forum "Big project" (Please feel free to email me in case the answer is not addressed in the forum at maximiliano.cubillos@polimi.it, after the holiday period)
- Submission deadline:  February 10th 00:00 hrs. No late admissions will be considered.

**Student names**


- Name 1
- Name 2
- Name 3


In [5]:
file_path = "test02.json"

In [7]:
# When using Colab, make sure you run this instruction beforehand
import importlib
import cffi
importlib.reload(cffi)

import json

with open(file_path, 'r') as f:
    data = json.load(f)

days = data["days"]

rooms = []
for room in data["rooms"]:
    rooms.append({
        "id": room["id"],
        "capacity": room["capacity"]
    })

#patients arrayList
patients = []
for patient in data["patients"]:
  patients.append({
      "id":  patient["id"],
      "length_of_stay" : patient["length_of_stay"],
      "release_date": patient["release_date"],
      "due_date": patient["due_date"],
      "incompatible_room_ids": patient["incompatible_room_ids"]
  })
  
#nurses arrayList
nurses = []
for nurse in data ["nurses"]:
  nurses.append({
      "id": nurse["id"],
      "working_shifts": nurse["working_shifts"]
  })

# Determine the dimensions of the matrix
num_patients = len(patients)
num_rooms = len(rooms)

# Initialize patients_rooms as a 2D list with default values of 1 (compatible)
patients_rooms = [[1 for _ in range(num_rooms)] for _ in range(num_patients)]

# Set incompatible entries to 0
for patient_index in range(num_patients):
    patient = patients[patient_index]
    for room_index in range(num_rooms):
        room = rooms[room_index]
        if room["id"] in patient["incompatible_room_ids"]:
           patients_rooms[patient_index][room_index] = 0

# Determine the dimensions of the matrix
num_nurses = len(nurses)

# Initialize nurses_days as a 2D list with default values of 1 (compatible)
nurses_days = [[1 for _ in range(days)] for _ in range(num_nurses)]

# Set incompatible entries to 0
for nurse_index in range(num_nurses):
    working_shift = nurses[nurse_index]["working_shifts"]
    values = []
    for i in working_shift:
        values.append(i["day"])
    for day in range(days):
        if day not in values:
            nurses_days[nurse_index][day] = 0

import mip

m = mip.Model()

#Decision variables

#x: a patient p stays in room r from a day in i and a day out o

x = {(p["id"], r["id"], i, o): m.add_var(var_type = mip.BINARY) for p in patients for r in rooms for i in range(days) for o in range(i, days)}

# y: a nurse n works on day d in room r
y = {(n["id"], r["id"], d): m.add_var(var_type = mip.BINARY) for n in nurses for r in rooms for d in range(days)}

#Objective function
m.objective = mip.minimize(
    mip.xsum(
        mip.xsum(
            mip.xsum(
                x[p["id"], r["id"], i, o] * (i - p["release_date"])
                for i in range(days) for o in range(i, days)
            ) for r in rooms
        ) for p in patients
    )
)
# Each patient can only be scheduled for a range of days and assigned to one room on that range.
for p in patients:
    m.add_constr(
        mip.xsum(
            x[p["id"], r["id"], i, o]
            for r in rooms
            for i in range(days)
            for o in range(i, days)
        ) == 1
    )

# 1. A patient’s length of stay must be respected, meaning they occupy a room for the full duration of their stay.
for p in patients:
    m.add_constr(
        mip.xsum(
            x[p["id"], r["id"], i, o]
            for r in rooms
            for i in range(days)
            for o in range(i, days)
            if o - i + 1 == p["length_of_stay"]
        ) == 1
    )
# 2. A room cannot exceed its capacity on any given day.
for d in range(days):
  for r in rooms:
    m.add_constr(mip.xsum(x[p["id"], r["id"], i, o] for p in patients for i in range(days) for o in range(i, days) if i <= d <= o ) <= r["capacity"])

# 3. Patients can only be assigned to rooms that they are compatible with.
for p in patients:
  for r in rooms:
    m.add_constr(mip.xsum(x[p["id"], r["id"], i, o] for i in range(days) for o in range(i, days)) <= patients_rooms[p["id"]][r["id"]])

# 4. Each occupied room must have exactly one assigned nurse per day.
# 4.a For each room and for each day the number of nurses must be greater or equal to 1 for each assigned room
for r in rooms:
    for d in range(days):
        for p in patients:
            m.add_constr(
            mip.xsum(y[n["id"], r["id"], d] for n in nurses) >=
            mip.xsum(x[p["id"], r["id"], i, o] for i in range(days) for o in range(i, days) if i <= d <= o)
            )
# 4.b There can be one nurse at most assigned to a room in a given day
for r in rooms:
    for d in range(days):
        m.add_constr(
            mip.xsum(y[n["id"], r["id"], d] for n in nurses) <= 1
            )
# 5. Nurses can only be assigned to rooms on days when they are available.
for n in nurses:
  for d in range(days):
    for r in rooms:
      m.add_constr(y[n["id"], r["id"], d] <= nurses_days[n["id"]][d])

# 6. Nurses can be assigned to a maximum of 3 different rooms in the same day.
for n in nurses:
  for d in range(days):
    m.add_constr( 3 >= mip.xsum(y[n["id"], r["id"], d] for r in rooms))

# 7 Admission days must be within the specified release and due dates.
for p in patients:
  m.add_constr(mip.xsum(mip.xsum(x[p["id"], r["id"], i, o] for i in range(days) for o in range(i, days) if i >= p["release_date"] and i <= p["due_date"]) for r in rooms) == 1)


m.optimize()
result = m.objective_value
if(result == None):
    print("No solution was found")
else:
    print(f"Total admission delay = {result}")

Total admission delay = 0.0
